## Data Requirements

### Receptionist
- `ReceptionistID (PK)`, FName, LName, PhoneNO, Day, Hours

### Patient
- `FileNo (PK)`, FName, LName, Age, PhoneNO, MedicalRecord, ReceptionistID

### Appointment
- `AppNo (PK)`, Date, Time, Status (Confirmed/Pending/Canceled), DoctorID, ReceptionistID

### Doctor
- `DoctorID (PK)`, FName, LName, RoomNo, PhoneNO, Department


## Transaction Requirements

**Data Entry:**
- Register a new patient
- Book/confirm appointments

**Data Update/Deletion:**
- Update medical record
- Update/delete appointments
- Update/delete patient phone

**Data Queries:**
- Appointments by date
- Doctor list by department
- Cancelled appointments
- Patients under 18
- Shared doctor patients
- Shift-specific receptionists
- Doctors in room 10


## Relational Schema

```sql
Doctor(DoctorID CHAR(8) PRIMARY KEY, FName VARCHAR(50), LName VARCHAR(50), RoomNO VARCHAR(2), PhoneNO VARCHAR(10), Department VARCHAR(20))
Appointment(AppNO CHAR(6) PRIMARY KEY, Date DATE, Time TIME, Status VARCHAR(10), DoctorID CHAR(8), ReceptionistID CHAR(8),
    FOREIGN KEY (DoctorID) REFERENCES Doctor(DoctorID),
    FOREIGN KEY (ReceptionistID) REFERENCES Reception(ReceptionistID)
)
Reception(ReceptionistID CHAR(8) PRIMARY KEY, FName VARCHAR(50), LName VARCHAR(50), PhoneNO VARCHAR(10), Day DATE, Hours TIME)
Patient(FileNO CHAR(6) PRIMARY KEY, FName VARCHAR(50), LName VARCHAR(50), Age INT, PhoneNO VARCHAR(10), MedicalRecord VARCHAR(100), ReceptionistID CHAR(8),
    FOREIGN KEY (ReceptionistID) REFERENCES Reception(ReceptionistID)
)
```


## Data Dictionary: Description of All Entities

| Entity      | Description                                                                                          | Occurrence                                                |
|-------------|------------------------------------------------------------------------------------------------------|-----------------------------------------------------------|
| Doctor      | A medical professional responsible for providing care to patients. A person who treats patients and has a valid medical certificate. | Assigned to none, one, or many appointments.              |
| Appointment | An arrangement for patient to meet a doctor at a particular time and date.                            | An appointment is assigned to one doctor.                 |
| Reception   | Manages patient appointments and records, acting as the first point of contact for patients.         | Each receptionist can register zero or many patients. They can book appointments for any patient with any doctor. |
| Patient     | A person who is receiving medical care in King Faisal hospital.                                      | Each patient can book zero or many appointments. Registered patients interact with receptionists to schedule appointments with doctors. |


## Data Dictionary: Description of All Relationships

| Entity Name  | Multiplicity | Relationship | Entity Name | Multiplicity |
|--------------|--------------|--------------|-------------|--------------|
| Appointment  | 0..*         | Assigned     | Doctor      | 1..1         |
| Reception    | 1..1         | Book         | Appointment | 0..*         |
| Reception    | 1..1         | Register     | Patient     | 0..*         |


## Data Dictionary: Description of All Attributes

| Entity   | Attribute      | Description                                       | Data Type | Length | Nulls | Multi-Valued | Default Value | Range                | PK  |
|----------|----------------|-------------------------------------------------|-----------|--------|-------|--------------|---------------|----------------------|-----|
| Reception| ReceptionistID | Unique id number assigned to receptionist        | Char      | 8      | No    | No           |               | 0-9                  | Yes |
|          | FName          | Receptionist's first name                         | Varchar   | 50     | Yes   | No           |               |                      |     |
|          | LName          | Receptionist's last name                          | Varchar   | 50     | Yes   | No           |               |                      |     |
|          | PhoneNO        | Receptionist's phone number                       | Varchar   | 10     | Yes   | No           |               | 0-9                  |     |
|          | Day            | The date of the day receptionist is on duty      | Date      |        | Yes   | No           |               | 01/01/2024 - 31/12/2024 |     |
|          | Hours          | Receptionist's working hours                      | Time      |        | Yes   | No           |               | 12:00am - 11:59pm    |     |

| Patient  | FileNo         | Unique number assigned to patient                 | Char      | 6      | No    | No           |               | 0-9                  | Yes |
|          | FName          | Patient's first name                              | Varchar   | 50     | Yes   | No           |               |                      |     |
|          | LName          | Patient's last name                               | Varchar   | 50     | Yes   | No           |               |                      |     |
|          | Age            | Patient's age                                    | Int       |        | Yes   | No           |               |                      |     |
|          | PhoneNO        | Patient's phone number                            | Varchar   | 10     | Yes   | No           |               | 0-9                  |     |
|          | MedicalRecord  | Record contains information about patient's previous visits | Varchar   | 100    | Yes   | No           |               |                      |     |

| Appointment | AppNo        | Unique number assigned to appointment             | Char      | 6      | No    | No           |               | 0-9                  | Yes |
|            | Date         | Appointment's date                                | Date      |        | Yes   | No           |               | 01/01/2024 - 31/12/2024 |     |
|            | Time         | Appointment time in hours                         | Time      |        | Yes   | No           |               | 12:00am - 11:59pm    |     |
|            | Status       | Appointment's visit status                        | Varchar   | 10     | Yes   | No           | 'Confirmed', 'pending', or 'canceled' |     |

| Doctor    | DoctorID       | Unique id number assigned to doctor               | Char      | 8      | No    | No           |               | 0-9                  | Yes |
|           | FName         | Doctor's first name                               | Varchar   | 50     | Yes   | No           |               |                      |     |
|           | LName         | Doctor's last name                                | Varchar   | 50     | Yes   | No           |               |                      |     |
|           | RoomNO        | Doctor's room number in the clinic                | Varchar   | 2      | Yes   | No           |               | 1-50                 |     |
|           | PhoneNO       | Doctor's phone number                             | Varchar   | 10     | Yes   | No           |               | 0-9                  |     |
|           | Department    | Doctor's specialty                               | Varchar   | 20     | Yes   | No           |               |                      |     |


## Sample Queries

```sql
-- 1. Display patients who are younger than 18
SELECT * FROM Patient WHERE Age < 18;

-- 2. Display canceled appointments
SELECT * FROM Appointment WHERE Status = 'Canceled';

-- 3. Display doctors in a specific department (example: Cardiology)
SELECT FName, LName FROM Doctor WHERE Department = 'Cardiology';

-- 4. List all doctors who have worked in room number 10
SELECT * FROM Doctor WHERE RoomNO = '10';

-- 5. Returns all appointments scheduled for October 22nd
SELECT * FROM Appointment WHERE Date = '2024-10-22';

-- 6. Display patients treated by the same doctor
SELECT DISTINCT p1.FName AS Patient1, p2.FName AS Patient2, a1.DoctorID
FROM Appointment a1
JOIN Appointment a2 ON a1.DoctorID = a2.DoctorID AND a1.AppNO != a2.AppNO
JOIN Patient p1 ON a1.FileNO = p1.FileNO
JOIN Patient p2 ON a2.FileNO = p2.FileNO
WHERE p1.FileNO != p2.FileNO;

-- 7. Display receptionists working a specific shift (example: Hours = '09:00:00')
SELECT * FROM Reception WHERE Hours = '09:00:00';


## Sample Query Descriptions

- Display patients who are younger than 18 years old.
- Display names of doctors in a certain department.
- List all doctors who have worked in room number 10.
- Returns all appointments scheduled for October 22nd.
